In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('download.csv')#.transpose()

data = data.drop(0, axis=0)

In [3]:
class_data = pd.DataFrame(0, columns=['sta', 'sol', 'dyn', 'date'], index=[i for i in range(len(data.columns)-1)])

for i, row in enumerate(data.iloc()):
    name = row['Nth day']

    if 'sta' in name or "/course_instance/158188" in name:
        class_data['sta'] += np.array([int(i) for i in row[1:].values])

    if 'sol' in name or "/course_instance/157231" in name:
        class_data['sol'] += np.array([int(i) for i in row[1:].values])
    
    if 'dyn' in name or "/course_instance/158221" in name:
        class_data['dyn'] += np.array([int(i) for i in row[1:].values])

dates = [dt.datetime(year=2024, month=10, day=4) + dt.timedelta(days=i) for i in range(len(class_data.index))]

class_data['date'] = dates

class_data = class_data.set_index('date')

day = dt.datetime(year=2024, month=10, day=6)

days = [[] for i in range(7)]

for i in range(7):
    temp = day + dt.timedelta(days=i)
    while temp < dt.datetime(year=2024, month=11, day=17):
        days[i].append(temp)
        temp += dt.timedelta(days=7)

days

[[datetime.datetime(2024, 10, 6, 0, 0),
  datetime.datetime(2024, 10, 13, 0, 0),
  datetime.datetime(2024, 10, 20, 0, 0),
  datetime.datetime(2024, 10, 27, 0, 0),
  datetime.datetime(2024, 11, 3, 0, 0),
  datetime.datetime(2024, 11, 10, 0, 0)],
 [datetime.datetime(2024, 10, 7, 0, 0),
  datetime.datetime(2024, 10, 14, 0, 0),
  datetime.datetime(2024, 10, 21, 0, 0),
  datetime.datetime(2024, 10, 28, 0, 0),
  datetime.datetime(2024, 11, 4, 0, 0),
  datetime.datetime(2024, 11, 11, 0, 0)],
 [datetime.datetime(2024, 10, 8, 0, 0),
  datetime.datetime(2024, 10, 15, 0, 0),
  datetime.datetime(2024, 10, 22, 0, 0),
  datetime.datetime(2024, 10, 29, 0, 0),
  datetime.datetime(2024, 11, 5, 0, 0),
  datetime.datetime(2024, 11, 12, 0, 0)],
 [datetime.datetime(2024, 10, 9, 0, 0),
  datetime.datetime(2024, 10, 16, 0, 0),
  datetime.datetime(2024, 10, 23, 0, 0),
  datetime.datetime(2024, 10, 30, 0, 0),
  datetime.datetime(2024, 11, 6, 0, 0),
  datetime.datetime(2024, 11, 13, 0, 0)],
 [datetime.datetime(

In [4]:
daily_avgs_no_quiz = pd.DataFrame(0, columns=['sta', 'sol', 'dyn'], index=[i+1 for i in range(7)])
daily_avgs_quiz = pd.DataFrame(0, columns=['sta', 'sol', 'dyn'], index=[i+1 for i in range(7)])

for i in range(7):
    for j, day in enumerate(days[i]):
        if not j % 2:
            
            daily_avgs_quiz.iloc[i]['sol'] += np.array(class_data.loc[class_data.index == day].values[0][1])
            daily_avgs_no_quiz.iloc[i]['dyn'] += np.array(class_data.loc[class_data.index == day].values[0][2])
        else:
            
            daily_avgs_no_quiz.iloc[i]['sol'] += np.array(class_data.loc[class_data.index == day].values[0][1])
            daily_avgs_quiz.iloc[i]['dyn'] += np.array(class_data.loc[class_data.index == day].values[0][2])
    
    for j, day in enumerate(days[i]):

        if j in [0, 2, 5]:
            if i == 0:
                print(day)
            daily_avgs_quiz.iloc[i]['sta'] += np.array(class_data.loc[class_data.index == day].values[0][0])
        else:
            daily_avgs_no_quiz.iloc[i]['sta'] += np.array(class_data.loc[class_data.index == day].values[0][0])

2024-10-06 00:00:00
2024-10-20 00:00:00
2024-11-10 00:00:00


In [5]:
for i, class_ in enumerate(['sta', 'sol', 'dyn']):
    r = pd.DataFrame(index=[i+1 for i in range(7)], columns=['quiz', 'noquiz'])
    r['quiz'] = daily_avgs_quiz[class_]
    r['noquiz'] = daily_avgs_no_quiz[class_]

    r.to_csv(f"{class_}.csv")

In [6]:
daily_avgs_no_quiz

,sta,sol,dyn
1,205,181,454
2,738,350,960
3,677,882,375
4,1218,1048,420
5,780,330,780
6,557,817,612
7,175,547,233


In [23]:
import plotly.express as px

for class_ in ['sta', 'sol', 'dyn']:
    graph_data = pd.read_csv(f'{class_}.csv')
    graph_data = graph_data.set_index(graph_data.columns[0])
    fig = px.bar(graph_data, x=graph_data.index, y=["quiz", "noquiz"], barmode='group')
    if class_ == "sol":
        fig.add_vline(x=4.5, line_width=3, line_dash="dash", line_color="red")
        fig.add_vline(x=7.5, line_width=3, line_dash="dash", line_color="red")
    if class_ == "sta":
        fig.add_vline(x=3.5, line_width=3, line_dash="dash", line_color="red")
        fig.add_vline(x=6.5, line_width=3, line_dash="dash", line_color="red")
    if class_ == "dyn":
        fig.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="red")
        fig.add_vline(x=5.5, line_width=3, line_dash="dash", line_color="red")
    fancy_name = "Dynamics" if class_ == 'dyn' else ('Solid Mechanics' if class_ == 'sol' else 'Statics')
    fig.update_layout(
        title=fancy_name,
        xaxis=dict(title="Day of week (1 = sunday)"),
        yaxis=dict(title="Total visitorship")
    )
    fig.show()

In [34]:
class_data

,sta,sol,dyn
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
56,NaN,NaN,NaN
57,NaN,NaN,NaN
58,NaN,NaN,NaN
59,NaN,NaN,NaN
